In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = 'v0'
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_00'
    dataset_name = 'RNA_DatasetBaseline'
    
    model_name = 'RNA_ModelV2'
    dim = 192
    depth = 12
    dim_head = 32
   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [4]:
df = pd.read_parquet(os.path.join(CFG.path,'train_data.parquet'))
ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='train',sn_train=False)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='train', mask_only=True,sn_train=False)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

0

In [ ]:
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)

epoch,train_loss,valid_loss,mae,time
0,0.141563,0.172369,0.184915,12:35


Better model found at epoch 0 with mae value: 0.1849148832085437.


In [ ]:
rnacomp.dataset